In [1]:
import cv2
import os
import pymysql
import numpy as np
from skimage import io
import mtcnn
import utils as utils
import InceptionResNetV1

Using TensorFlow backend.


In [5]:
from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

In [ ]:
with PyCallGraph(output=GraphvizOutput()):
    mtcnn_model = mtcnn()
    # threshold
    threshold = [0.5,0.8,0.9]
    # load facenet
    # detected face transfer to 128-dimensional vector
    facenet_model = InceptionResNetV1()
    # model.summary()
    model_path = './model_data/facenet_keras.h5'
    facenet_model.load_weights(model_path)
    face_list = os.listdir("face_dataset")
    known_face_encodings=[]
    known_face_names=[]
    nametest=[]
    ilist=[]
    i=0;
    for face in face_list:
        name = face.split(".")[0]
        print(name)
        nametest.append(name)
        img = cv2.imread("./face_dataset/"+name+".jpg")
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        rectangles = mtcnn_model.detectFace(img, threshold)
        # convert to rectangle
        rectangles = utils.rect2square(np.array(rectangles))
        # facenet input a 160*160 picture
        rectangle = rectangles[0]
        # mark the landmark
        landmark = (np.reshape(rectangle[5:15],(5,2)) - np.array([int(rectangle[0]),int(rectangle[1])]))/(rectangle[3]-rectangle[1])*160
        crop_img = img[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])]
        crop_img = cv2.resize(crop_img,(160,160))
        new_img,_ = utils.Alignment_1(crop_img,landmark)
        new_img = np.expand_dims(new_img,0)
        # transfer the detected face into facenet, realize the 128-dimensional feature vector extraction
        face_encoding = utils.calc_128_vec(facenet_model,new_img)
        known_face_encodings.append(face_encoding)
        known_face_names.append(name)



In [10]:
face_list = os.listdir("face_dataset")
known_face_encodings=[]
known_face_names=[]
nametest=[]
ilist=[]
i=0;
for face in face_list:
    name = face.split(".")[0]
    print(name)
    nametest.append(name)
    img = cv2.imread("./face_dataset/"+name+".jpg")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    rectangles = mtcnn_model.detectFace(img, threshold)
    # convert to rectangle
    rectangles = utils.rect2square(np.array(rectangles))
    # facenet input a 160*160 picture
    rectangle = rectangles[0]
    # mark the landmark
    landmark = (np.reshape(rectangle[5:15],(5,2)) - np.array([int(rectangle[0]),int(rectangle[1])]))/(rectangle[3]-rectangle[1])*160
    crop_img = img[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])]
    crop_img = cv2.resize(crop_img,(160,160))
    new_img,_ = utils.Alignment_1(crop_img,landmark)
    new_img = np.expand_dims(new_img,0)
    # transfer the detected face into facenet, realize the 128-dimensional feature vector extraction
    face_encoding = utils.calc_128_vec(facenet_model,new_img)
    known_face_encodings.append(face_encoding)
    known_face_names.append(name)
    # the number of the image, used to store the picture
    ilist.append(i)
    i=i+1
    # First convert the data into a string type that can be stored in the database (float32->list->string)
    tlist=face_encoding.tolist()  #Convert a 128-dimensional vector into a list
    tlist_str=str(tlist)
    #update image
    db = pymysql.connect(host = 'mydbinstance.ctt6qfpp9mdh.eu-west-3.rds.amazonaws.com',
                     port = 3306,
                     user = 'husiyu',
                     passwd = 'husiyu123',
                     database = 'dbname',
                     charset = 'utf8')
    #获get cursor
    cur = db.cursor()
    sql = "Update dbname.images set out1 = '"+ tlist_str +"' where id="+str(i)+";"  # need to transfer it to float32 afterwards
    cur.execute(sql)
    db.commit()
    cur.close()
    db.close()

obama
xi


In [171]:
#This module code is responsible for putting the picture into the database! ! ! ! ! ! ! ! ! !
fin = open("./face_dataset/xi.jpg",'rb')   #'Add rb' to read the picture as binary. Only one updated here
imgtwo = fin.read()                 #Read binary data into img
fin.close()
db = pymysql.connect(host = 'mydbinstance.ctt6qfpp9mdh.eu-west-3.rds.amazonaws.com',
                     port = 3306,
                     user = 'husiyu',
                     passwd = 'husiyu123',
                     database = 'dbname',
                     charset = 'utf8')
    #get cursor
cur = db.cursor()
sql = "Update dbname.images set data = %s where id=1;"  # need to transfer it to float32 afterwards
args = (imgtwo)                              
cur.execute(sql,args)  
db.commit()  #update db
cur.close()
db.close()

In [173]:
#read image from database
db = pymysql.connect(host = 'mydbinstance.ctt6qfpp9mdh.eu-west-3.rds.amazonaws.com',
                     port = 3306,
                     user = 'husiyu',
                     passwd = 'husiyu123',
                     database = 'dbname',
                     charset = 'utf8')
#get cursor
cur = db.cursor()
sql = "SELECT data FROM dbname.images where id=1;"
cur.execute(sql)
fout = open('image.jpg','wb')
fout.write(cur.fetchone()[0])
fout.close()
db.commit()
cur.close()
db.close()
from PIL import Image
im = Image.open('image.jpg')


In [145]:
#Read the 128-dimensional vector data in the database (the string stored in the database
#, so you have to convert the matrix) and convert it to a matrix
db = pymysql.connect(host = 'mydbinstance.ctt6qfpp9mdh.eu-west-3.rds.amazonaws.com',
                     port = 3306,
                     user = 'husiyu',
                     passwd = 'husiyu123',
                     database = 'dbname',
                     charset = 'utf8')
#get cursor
cur = db.cursor()
sql = "SELECT out1 FROM dbname.images;"  # need to transfer it to float32 afterwards
cur.execute(sql)
result=cur.fetchall()
db.commit()
cur.close()
db.close()